# Grab USDA county-level maize data

In [1]:
import requests
import pandas as pd
import numpy as np
import geopandas as gp

api_key = '210BA222-FC6E-3FB2-B4D7-DA2DAA1CC829'

In [2]:
state_alphas = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA",
                "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD",
                "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ",
                "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC",
                "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

needless_info = ["CV (%)", "agg_level_desc", "asd_desc", "begin_code", "asd_code", 
                 "class_desc", "commodity_desc", "congr_district_code", "state_name",
                 "country_code", "country_name", "domain_desc",
                 "domaincat_desc", "end_code", "freq_desc", "group_desc",
                 "load_time", "location_desc", "prodn_practice_desc",
                 "reference_period_desc", "region_desc", "sector_desc",
                 "short_desc", "source_desc", "statisticcat_desc", "unit_desc",
                 "util_practice_desc", "watershed_code", "watershed_desc",
                 "week_ending", "zip_5", "county_ansi", "state_ansi"]


def get_corn(states, yields=True):
    """
    Grabs county-level corn data from USDA API.
    Input: states = list of U.S. state codes
           yields = boolean, if True return yields in bushels/acre, else return area in acres
    Output: pandas dataframe
    """
    data = pd.DataFrame()
    for state in states:
        print("Now grabbing: " + state)
        if yields:
            dat = requests.get("http://quickstats.nass.usda.gov/api/api_GET/?key=" + api_key + "&source_desc=SURVEY&sector_desc=CROPS&group_desc=FIELD CROPS&commodity_desc=CORN&statisticcat_desc=YIELD&util_practice_desc=GRAIN&unit_desc=BU / ACRE&agg_level_desc=COUNTY&year__GE=1950&state_alpha=" + state)
        else:
            dat = requests.get("http://quickstats.nass.usda.gov/api/api_GET/?key=" + api_key + "&source_desc=SURVEY&sector_desc=CROPS&group_desc=FIELD CROPS&commodity_desc=CORN&statisticcat_desc=AREA HARVESTED&util_practice_desc=GRAIN&unit_desc=ACRES&agg_level_desc=COUNTY&year__GE=1950&state_alpha=" + state)
        if dat.status_code == 200:
            print("Data grabbed from USDA successfully...")
            dat = dat.json()
            dat = pd.DataFrame(dat["data"])
            data = pd.concat([data, dat], ignore_index=True)
            print("Filled!")
        else:
            print("Data grabbed from USDA unsuccessfully. Error code "
                  + str(dat.status_code) + ". Skipping.")
    return data.drop(columns=needless_info)

In [3]:
# Get data
usda_county_yield = get_corn(state_alphas)
print('\n\n\n Now for the areas...\n\n\n')
usda_county_area = get_corn(state_alphas, yields=False)

Now grabbing: AL
Data grabbed from USDA successfully...
Filled!
Now grabbing: AK
Data grabbed from USDA unsuccessfully. Error code 400. Skipping.
Now grabbing: AZ
Data grabbed from USDA successfully...
Filled!
Now grabbing: AR
Data grabbed from USDA successfully...
Filled!
Now grabbing: CA
Data grabbed from USDA successfully...
Filled!
Now grabbing: CO
Data grabbed from USDA successfully...
Filled!
Now grabbing: CT
Data grabbed from USDA unsuccessfully. Error code 400. Skipping.
Now grabbing: DE
Data grabbed from USDA successfully...
Filled!
Now grabbing: FL
Data grabbed from USDA successfully...
Filled!
Now grabbing: GA
Data grabbed from USDA successfully...
Filled!
Now grabbing: HI
Data grabbed from USDA unsuccessfully. Error code 400. Skipping.
Now grabbing: ID
Data grabbed from USDA successfully...
Filled!
Now grabbing: IL
Data grabbed from USDA successfully...
Filled!
Now grabbing: IN
Data grabbed from USDA successfully...
Filled!
Now grabbing: IA
Data grabbed from USDA successful

In [4]:
# Merge and tidy
usda_county_yield.rename(columns = {'Value':'yield', 'state_fips_code':'state'}, inplace=True)
usda_county_yield['fips'] = usda_county_yield['state'] + usda_county_yield['county_code']
usda_county_yield.drop(columns = ['county_code'], inplace=True)

usda_county_area.rename(columns = {'Value':'area', 'state_fips_code':'state'}, inplace=True)
usda_county_area['fips'] = usda_county_area['state'] + usda_county_area['county_code']
usda_county_area.drop(columns = ['county_code'], inplace=True)

usda_county_all = pd.merge(usda_county_yield.drop_duplicates(subset=['fips','year']).query('county_name != "OTHER (COMBINED) COUNTIES"'),
                           usda_county_area.drop_duplicates(subset=['fips','year']).query('county_name != "OTHER (COMBINED) COUNTIES"'),
                           how='inner')

In [5]:
# County shapefile with lat, lon of centroid
county_shapefile = gp.read_file('../other/plotting_tools/counties_contig_plot.shp')
county_shapefile_lat_lon = county_shapefile.to_crs("EPSG:4326")
county_shapefile_lat_lon['lat'] = county_shapefile_lat_lon['geometry'].geometry.centroid.y
county_shapefile_lat_lon['lon'] = county_shapefile_lat_lon['geometry'].geometry.centroid.x

/Users/davidlafferty/opt/anaconda3/envs/BCSD_CornYields_UQ/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  after removing the cwd from sys.path.
/Users/davidlafferty/opt/anaconda3/envs/BCSD_CornYields_UQ/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  """


In [6]:
# Merge
county_all = pd.merge(county_shapefile_lat_lon.drop(columns='geometry'), usda_county_all, on = 'fips', how='inner')

In [7]:
# To numeric
county_all['yield'] = county_all['yield'].astype(float)
county_all['area'] = county_all['area'].apply(lambda x: float(x.replace(",","")) if type(x) == str else x)
# Log yield
county_all['log_yield'] = county_all['yield'].apply(lambda x: np.log(x) if x > 0.0 else np.nan)

In [13]:
county_all

,fips,lat,lon,year,state,state_alpha,yield,county_name,area,log_yield
0,31039,41.916403,-96.787400,2020,31,NE,179.7,CUMING,156500.0,5.191289
1,31039,41.916403,-96.787400,2019,31,NE,204.6,CUMING,148900.0,5.321057
2,31039,41.916403,-96.787400,2018,31,NE,200.0,CUMING,151100.0,5.298317
3,31039,41.916403,-96.787400,2017,31,NE,203.9,CUMING,149000.0,5.317630
4,31039,41.916403,-96.787400,2016,31,NE,197.8,CUMING,153000.0,5.287256
...,...,...,...,...,...,...,...,...,...,...
145276,04001,35.395603,-109.488854,1969,04,AZ,13.0,APACHE,5200.0,2.564949
145277,04001,35.395603,-109.488854,1968,04,AZ,15.0,APACHE,2700.0,2.708050
145278,04001,35.395603,-109.488854,1967,04,AZ,18.1,APACHE,3500.0,2.895912
145279,04001,35.395603,-109.488854,1966,04,AZ,21.8,APACHE,3700.0,3.081910


In [14]:
# Save
county_all.to_csv("../data/usda/maize_county_yield_area.csv", index=False)

In [9]:
############# NOT NEEDED
# # Build index structure for final dataframe (including years)
# all_fips = county_all['fips'].unique()
# years = np.arange(1950, 2021, 1)

# all_fips = [[fips]*len(years) for fips in all_fips]
# all_fips = np.ndarray.flatten(np.asarray(all_fips))

# years = [years] * len(all_fips)
# years = np.ndarray.flatten(np.asarray(years))

# tuples = list(zip(*[all_fips, years]))

# index = pd.MultiIndex.from_tuples(tuples)

# # Build empty dataframe with complete indexing
# all_index = pd.DataFrame(index = index)
# all_index.index.names = ["fips", "year"]

# # Merge yields so that empty county/year pairings are "NaN"
# county_all_indexed = pd.merge(all_index, county_all, on = ['fips', 'year'],  how = "outer")